# Subreddit Stats

In [1]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as md
import numpy as np
import praw
from psaw import PushshiftAPI
import datetime
import yaml
import time
from sortedcontainers import SortedDict
from collections import OrderedDict

## Parameters

In [2]:
start_epoch = int(datetime.datetime(2018, 1, 1).timestamp())
start_subs = 680       # number of subs at start_epoch (needed if before 2018/3/15, see redditmetrics.com)
end_epoch = int(datetime.datetime(2019, 1, 1).timestamp())
max_posts = 10000      # max number of posts to crawl, None for unlimited
sub = 'dreamcatcher'   # subreddit to crawl

## Get Data

In [3]:
class Comment:
    def __init__(self):
        self.body = ''
        self.author = ''
        self.score = 0
        self.ts = 0
        self.url = ''

class Post:
    def __init__(self):
        self.title = ''
        self.author = ''
        self.ts = 0
        self.score = 0
        self.comments = []
        self.url = ''

In [ ]:
stream = open('client_info.txt', 'r')
doc = yaml.load(stream)

r = praw.Reddit(client_id=doc['client_id'],
                client_secret=doc['client_secret'],
                user_agent='test'
                )
api_ps_praw = PushshiftAPI(r)
api_ps = PushshiftAPI()
post_ids = api_ps_praw.search_submissions(after=start_epoch,
                                  before=end_epoch,
                                  subreddit=sub,
                                  filter=['url', 'author', 'title', 'subreddit'],
                                  limit=max_posts)
subs = api_ps.search_submissions(after=start_epoch,
                                  before=end_epoch,
                                  subreddit=sub,
                                  filter=['subreddit', 'id', 'subreddit_subscribers'],
                                  limit=max_posts)
id2subs = {}

for s in subs:
    if hasattr(s, 'subreddit_subscribers'):
        num_subs = s.subreddit_subscribers
    else:
        num_subs = -1    
    id2subs[s.id] = num_subs
            
subcount = SortedDict()
posts = []
prev_info_len = 1

for i, post_id in enumerate(post_ids):
    submission = praw.models.Submission(r, id=post_id)
    ts = int(submission.created_utc)
    subcount[ts] = id2subs[post_id.id]
    submission.comments.replace_more()
    comment_ids = submission.comments.list()
    c = []
    
    for comment_id in comment_ids:
        comment = praw.models.Comment(r, id=comment_id)
        temp = Comment()
        temp.body = comment.body
        temp.author = comment.author
        temp.ts = comment.created_utc
        temp.score = comment.score
        temp.url = comment.permalink
        c.append(temp)
    
    ts_str = datetime.datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
    temp = Post()
    temp.title = submission.title
    temp.author = submission.author
    temp.ts = submission.created_utc
    temp.score = submission.score
    temp.comments = c
    temp.url = submission.permalink
    posts.append(temp)
    
    title = (temp.title[:47] + '...') if len(temp.title) > 47 else temp.title
    info = '%05d: %s --- %s' % (i, ts_str, title)
    clear = ' ' * prev_info_len
    print('%s' % clear, end='\r', flush=True)
    prev_info_len = len(info)
    print('%s' % info, end='\r', flush=True)
    
num_posts = i+1
    
print('\ndone')

## Visualize Data

### Subscriber Growth

In [ ]:
timestamps = np.zeros(len(subcount))
subs = np.zeros(len(subcount))
no_sub_data_end = -1
for i, ts in enumerate(subcount):
    timestamps[i] = ts
    subs[i] = subcount[ts]
    if subs[i] == -1:
        no_sub_data_end = i
        
    
if start_subs is not None:
    if no_sub_data_end < len(subs)-1:
        subs[:no_sub_data_end+1] = np.linspace(start_subs, subs[no_sub_data_end+1], no_sub_data_end+1)
        
for i in range(no_sub_data_end):
    subcount[timestamps[i]] = subs[i]
    
dates = [datetime.datetime.fromtimestamp(ts) for ts in timestamps]

plt.figure(figsize=(12,10))
plt.xticks(rotation=45)
ax = plt.gca()
xfmt = md.DateFormatter('%y-%m')
ax.xaxis.set_major_formatter(xfmt)
plt.plot(dates, subs)
plt.xlabel('date')
plt.ylabel('subscriber count')
plt.title('/r/%s Subscriber Growth' % (sub))
plt.show()

### Top Posts

In [ ]:
adj_scores = np.zeros((len(posts)), dtype=float)
for i, post in enumerate(posts):
    ts = int(post.ts)
    adj_scores[i] = (float(post.score) / subcount[ts])


top_idx = (-adj_scores).argsort()[:10]

for i, idx in enumerate(top_idx):
    post = posts[idx]
    ts = int(post.ts)
    ts_str = datetime.datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
    print('top post #%d' % (i+1))
    print('title:          %s' % (post.title))
    print('author:         %s' % (post.author))
    print('time:           %s' % (ts_str))
    print('score:          %d' % (post.score))
    print('adjusted score: %0.2f' % (float(post.score) / subcount[int(post.ts)]))
    print('comments:       %d' % (len(post.comments)))
    print('permalink:      https://www.reddit.com%s' % (post.url))
    print()

### Top Comments

In [ ]:
all_comments = []
comment_ts2post_ts = {}
for i, post in enumerate(posts):
    ts = int(post.ts)
    for comment in post.comments:
        all_comments.append(comment)
        if comment.ts not in comment_ts2post_ts:
            comment_ts2post_ts[comment.ts] = ts

adj_scores = np.zeros((len(all_comments)), dtype=float)
for i in range(len(all_comments)):
    ts = int(all_comments[i].ts)
    adj_scores[i] = float(all_comments[i].score) / subcount[comment_ts2post_ts[ts]]
    
top_idx = (-adj_scores).argsort()[:10]

for i, idx in enumerate(top_idx):
    comment = all_comments[idx]
    body = comment.body.replace('\n', ' ')
    body = (body[:97] + '...') if len(body) > 97 else body
    ts = int(comment.ts)
    ts_str = datetime.datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
    print('top comment #%d' % (i+1))
    print('comment:        %s' % (body))
    print('author:         %s' % (comment.author))
    print('time:           %s' % (ts_str))
    print('score:          %d' % (comment.score))
    print('adjusted score: %f' % (adj_scores[idx]))
    print('permalink:      https://www.reddit.com%s' % (comment.url))
    print()

### Top Users

In [ ]:
users = OrderedDict()
for i, post in enumerate(posts):
    ts = int(post.ts)
    if post.author == None:
        continue
    if post.author not in users:
        users[post.author] = {}
        users[post.author]['score'] = 0
        users[post.author]['adj_score'] = 0
        users[post.author]['posts'] = 0
        users[post.author]['comment_score'] = 0
        users[post.author]['adj_comment_score'] = 0
        users[post.author]['comments'] = 0
    users[post.author]['score'] += post.score
    users[post.author]['adj_score'] += (float(post.score) / subcount[ts])
    users[post.author]['posts'] += 1
    for comment in post.comments:
        if comment.author not in users:
            users[comment.author] = {}
            users[comment.author]['score'] = 0
            users[comment.author]['adj_score'] = 0
            users[comment.author]['posts'] = 0
            users[comment.author]['comment_score'] = 0
            users[comment.author]['adj_comment_score'] = 0
            users[comment.author]['comments'] = 0
        users[comment.author]['comment_score'] += comment.score
        users[comment.author]['adj_comment_score'] += (float(post.score) / subcount[ts])
        users[comment.author]['comments'] += 1

#### Top Posters

In [ ]:
sorted_users = OrderedDict(sorted(users.items(), key=lambda x: x[1]['adj_score'], reverse=True))

for i, key in enumerate(sorted_users):
    if i >= 10:
        break
    print('top poster #%d' % (i+1))
    print('user:                  %s' % (key))
    print('total number of posts: %d' % (sorted_users[key]['posts']))
    print('total score:           %d' % (sorted_users[key]['score']))
    print('total adjusted score:  %f' % (sorted_users[key]['adj_score']))
    print()

#### Top Commenters

In [ ]:
sorted_users = OrderedDict(sorted(users.items(), key=lambda x: x[1]['adj_comment_score'], reverse=True))

for i, key in enumerate(sorted_users):
    if i >= 10:
        break
    print('top commenter #%d' % (i+1))
    print('user:                          %s' % (key))
    print('total number of comments:      %d' % (sorted_users[key]['comments']))
    print('total comment score:           %d' % (sorted_users[key]['comment_score']))
    print('total adjusted comment score:  %f' % (sorted_users[key]['adj_comment_score']))
    print()